In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/fin_tech')

In [3]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.0 MB/s eta 0:00:00


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
import category_encoders as ce
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

In [5]:
train_df = pd.read_csv('train_fin.csv')
test_df = pd.read_csv('test_fin.csv')

In [6]:
y = train_df['MIS_Status']
categorical_columns = [
    'RevLineCr', 'LowDoc', 'City', 'State', 'BankState', 'NewExist', 'UrbanRural'
]

In [7]:
def target_enc(col, X, y, test):
    kf = KFold(n_splits=5)
    encoded_features = []

    for train_idx, val_idx in kf.split(X, y):
        X_train_, X_valid_ = X.iloc[train_idx], X.iloc[val_idx]
        y_train_ = y.iloc[train_idx]

        target_encoder = ce.TargetEncoder()
        target_encoder.fit(X_train_[col], y_train_)

        X_valid_[f'target_{col}'] = target_encoder.transform(X_valid_[col])
        encoded_features.append(X_valid_)

    encoded_df = pd.concat(encoded_features).sort_index()
    df_with_encoded = pd.merge(X, encoded_df[[f'target_{col}']], left_index=True, right_index=True, how='left')

    target_encoder = ce.TargetEncoder()
    target_encoder.fit(df_with_encoded[col], y)

    test[f'target_{col}'] = target_encoder.transform(test[col])

    df_with_encoded.drop(col, axis=1, inplace=True)
    test.drop(col, axis=1, inplace=True)

    return df_with_encoded, test

In [8]:
for col in categorical_columns:
    train_df, test_df = target_enc(col, train_df, y, test_df)

<ipython-input-7-85c695773bca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid_[f'target_{col}'] = target_encoder.transform(X_valid_[col])
<ipython-input-7-85c695773bca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid_[f'target_{col}'] = target_encoder.transform(X_valid_[col])
<ipython-input-7-85c695773bca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

<ipython-input-7-85c695773bca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid_[f'target_{col}'] = target_encoder.transform(X_valid_[col])
<ipython-input-7-85c695773bca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid_[f'target_{col}'] = target_encoder.transform(X_valid_[col])
<ipython-input-7-85c695773bca>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [13]:
train_df.to_csv('encoded_train_fin.csv', index=False)
test_df.to_csv('encoded_test_fin.csv', index=False)

In [14]:
#KNN欠損地補完
train_df = pd.read_csv('encoded_train_fin.csv')
test_df = pd.read_csv('encoded_test_fin.csv')

numeric_columns = train_df.select_dtypes(include=[np.number]).columns.tolist()
if 'MIS_Status' in numeric_columns:
    numeric_columns.remove('MIS_Status')

numeric_train_df = train_df[numeric_columns + ['MIS_Status']]
numeric_test_df = test_df[numeric_columns]

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

numeric_train_df_imputed = pd.DataFrame(imputer.fit_transform(numeric_train_df.drop(columns=['MIS_Status'])), columns=numeric_columns)
numeric_train_df_imputed['MIS_Status'] = train_df['MIS_Status']  # ターゲット変数を戻す

# テストデータセットに対してtransformを適用
numeric_test_df_imputed = pd.DataFrame(imputer.transform(numeric_test_df), columns=numeric_columns)

numeric_train_df_imputed.to_csv('imputed_encoded_train_fin.csv', index=False)
numeric_test_df_imputed.to_csv('imputed_encoded_test_fin.csv', index=False)

## チュートリアル

In [ ]:
def process_date_columns(df, date_columns):
    """
    指定されたデータフレームの複数の日付カラムを年、月、日に分割する関数。

    Args:
    df (pandas.DataFrame): 処理するデータフレーム。
    date_columns (list of str): 日付データが含まれているカラムの名前のリスト。

    Returns:
    pandas.DataFrame: 変換後のデータフレーム。
    """
    for date_column in date_columns:
        # 日付カラムをdatetimeオブジェクトに変換
        df[date_column] = pd.to_datetime(df[date_column], errors='coerce')

        # 年、月、日のカラムを追加
        df[date_column + '_Year'] = df[date_column].dt.year
        df[date_column + '_Month'] = df[date_column].dt.month
        df[date_column + '_Day'] = df[date_column].dt.day

        # 元の日付カラムを削除
        df = df.drop(date_column, axis=1)

    return df

In [ ]:
train = process_date_columns(train, ['DisbursementDate', 'ApprovalDate'])
test = process_date_columns(test, ['DisbursementDate', 'ApprovalDate'])



In [ ]:
train.head()

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,MIS_Status,...,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,DisbursementDate_Year,DisbursementDate_Month,DisbursementDate_Day,ApprovalDate_Year,ApprovalDate_Month,ApprovalDate_Day
0,0,163,21,1.0,0,0,1,N,N,1,...,80000.00,80000.00,68000.00,0,1998.0,1.0,31.0,2006,9,22
1,1,84,6,1.0,4,0,0,0,N,1,...,287000.00,287000.00,229600.00,0,1993.0,10.0,31.0,1992,6,30
2,2,242,45,1.0,4,90,0,N,N,1,...,31983.00,30000.00,15000.00,1,2001.0,8.0,31.0,2001,4,18
3,3,237,4,1.0,0,0,0,N,N,1,...,229000.00,229000.00,229000.00,0,2007.0,8.0,31.0,2003,10,6
4,4,184,0,1.0,0,0,0,N,N,1,...,525000.00,525000.00,393750.00,0,1983.0,6.0,8.0,1999,12,17


In [ ]:
test.head(10)

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,Sector,...,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,DisbursementDate_Year,DisbursementDate_Month,DisbursementDate_Day,ApprovalDate_Year,ApprovalDate_Month,ApprovalDate_Day
0,42307,5,2,1.0,1,0,0,0.426724,0.907907,23,...,25000.00,25000.00,21250.00,1,2004.000000,7.00000,31.000000,2007,8,6
1,42308,235,13,1.0,9,14,77725,0.813273,0.907907,44,...,15000.00,15000.00,7500.00,0,1995.000000,1.00000,11.000000,2004,3,8
2,42309,31,5,2.0,0,0,0,0.924035,0.898570,56,...,28000.00,28000.00,23800.00,1,2001.014829,6.60857,26.629781,2007,2,27
3,42310,120,4,1.0,0,1,0,0.813273,0.907907,62,...,7500.00,7500.00,6375.00,2,1999.000000,4.00000,30.000000,1997,12,19
4,42311,63,13,1.0,0,8,1,0.924035,0.907907,42,...,91000.00,93000.00,93000.00,0,2005.000000,12.00000,31.000000,2009,7,10
5,42312,83,2,1.0,17,8,80555,0.892825,0.894448,23,...,100000.00,100000.00,50000.00,2,1994.000000,10.00000,31.000000,1994,11,30
6,42313,81,3,1.0,0,0,1,0.813273,0.907907,42,...,5000.00,5000.00,4250.00,1,2011.000000,4.00000,5.000000,2004,12,9
7,42314,32,9,1.0,1,3,0,0.924035,0.907907,42,...,42000.00,42000.00,33600.00,1,2010.000000,2.00000,1.000000,2004,8,26
8,42315,83,13,1.0,4,9,0,0.924035,0.907907,42,...,58547.00,30000.00,15000.00,1,2009.000000,2.00000,28.000000,2005,12,13
9,42316,15,0,2.0,0,0,0,0.924035,0.907907,0,...,50000.00,50000.00,40000.00,0,2003.000000,11.00000,30.000000,2006,10,27


In [ ]:
sample_submit = pd.read_csv('sample_submission.csv', index_col=0, header=None)

In [ ]:
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']]= \
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))
  #testデータも同様に変換します
test[['DisbursementGross',  'GrAppv', 'SBA_Appv']]= \
test[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', '')).astype(float)

train.head()

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,MIS_Status,...,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,DisbursementDate_Year,DisbursementDate_Month,DisbursementDate_Day,ApprovalDate_Year,ApprovalDate_Month,ApprovalDate_Day
0,0,163,21,1.0,0,0,1,0.924035,0.907907,1,...,80000.00,80000.00,68000.00,0,1998.0,1.0,31.0,2006,9,22
1,1,84,6,1.0,4,0,0,0.892825,0.907907,1,...,287000.00,287000.00,229600.00,0,1993.0,10.0,31.0,1992,6,30
2,2,242,45,1.0,4,90,0,0.924035,0.907907,1,...,31983.00,30000.00,15000.00,1,2001.0,8.0,31.0,2001,4,18
3,3,237,4,1.0,0,0,0,0.924035,0.907907,1,...,229000.00,229000.00,229000.00,0,2007.0,8.0,31.0,2003,10,6
4,4,184,0,1.0,0,0,0,0.924035,0.907907,1,...,525000.00,525000.00,393750.00,0,1983.0,6.0,8.0,1999,12,17


In [ ]:
train.fillna(train.mean(), inplace=True)

<ipython-input-37-d7386570b72f>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.fillna(train.mean(), inplace=True)


In [ ]:
test.fillna(test.mean(), inplace=True)

<ipython-input-53-177d09d2dfec>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test.fillna(test.mean(), inplace=True)


In [ ]:
# ターゲットエンコーディングを適用するカテゴリ変数を選択
categorical_columns_for_encoding = ['RevLineCr', 'LowDoc', 'City', 'State', 'BankState']

# ターゲットエンコーディングの実施
for col in categorical_columns_for_encoding:
    # 各カテゴリについて、ターゲット変数の平均を計算
    target_mean = train.groupby(col)['MIS_Status'].mean()
    # 訓練データにエンコーディングを適用
    train[col] = train[col].map(target_mean)
    # テストデータにもエンコーディングを適用（存在しないカテゴリはNaNになる）
    test[col] = test[col].map(target_mean)


In [ ]:
# 予測するターゲット変数
target = 'MIS_Status'

In [ ]:
X = train.drop(target,axis=1)
y = train[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# データセットとデータローダーの設定
X_train_tensor = torch.tensor(X_train.values.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.int64))
X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
y_val_tensor = torch.tensor(y_val.values.astype(np.int64))

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# ニューラルネットワークモデルの定義
class NeuralNet(nn.Module):
    def __init__(self, num_features):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(num_features, 64)
        self.layer2 = nn.Linear(64, 64)
        self.output_layer = nn.Linear(64, 3)  # 3つのクラスに分類

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

In [ ]:
model = NeuralNet(X_train_tensor.shape[1])

In [ ]:
# 損失関数とオプティマイザーの設定
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [ ]:
from sklearn.metrics import f1_score

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.numpy())

    val_f1 = f1_score(all_labels, all_predictions, average='macro')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation F1 Score (Macro): {val_f1}')


Epoch 1/30, Loss: 0.25552862882614136, Validation F1 Score (Macro): 0.47382166397214276
Epoch 2/30, Loss: 0.3255844712257385, Validation F1 Score (Macro): 0.47382166397214276
Epoch 3/30, Loss: 0.466668963432312, Validation F1 Score (Macro): 0.47382166397214276
Epoch 4/30, Loss: 0.2555916905403137, Validation F1 Score (Macro): 0.47382166397214276
Epoch 5/30, Loss: 0.36057326197624207, Validation F1 Score (Macro): 0.47382166397214276
Epoch 6/30, Loss: 0.43054530024528503, Validation F1 Score (Macro): 0.47382166397214276
Epoch 7/30, Loss: 0.3604623079299927, Validation F1 Score (Macro): 0.47382166397214276
Epoch 8/30, Loss: 0.14915497601032257, Validation F1 Score (Macro): 0.47382166397214276
Epoch 9/30, Loss: 0.29166415333747864, Validation F1 Score (Macro): 0.47382166397214276
Epoch 10/30, Loss: 0.36071401834487915, Validation F1 Score (Macro): 0.47382166397214276
Epoch 11/30, Loss: 0.3604018986225128, Validation F1 Score (Macro): 0.47382166397214276
Epoch 12/30, Loss: 0.256059199571609

In [ ]:
X_test_tensor = torch.tensor(test.values.astype(np.int64))

In [ ]:
X_test_tensor = X_test_tensor.float()

In [ ]:
# テストデータに対する予測
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predictions = torch.max(test_outputs, 1)

In [ ]:
# 予測結果をデータフレームに変換
predictions_df = pd.DataFrame({
    'ID': sample_submission.iloc[:, 0],  # sample_submission の最初のカラムをIDとして使用
    'Prediction': test_predictions.numpy()  # モデルの予測結果
})
# 提出ファイルとしてCSVに保存
submission_file_path = '/mnt/data/my_submission.csv'
predictions_df.to_csv(submission_file_path, index=False, header=False)  #ヘッダーなしで保存

# lightGBM

In [ ]:
numeric_columns = train.select_dtypes(include=['number']).columns
numeric_columns

Index(['Unnamed: 0', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'MIS_Status', 'Sector', 'ApprovalFY', 'UrbanRural'],
      dtype='object')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
  # train[std_columns] の相関行列を求める
corr_matrix = train[numeric_columns].corr()

  # 相関行列を図示
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Standardized Columns')
plt.show()

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_numeric = train[numeric_columns]

# データの準備
X = train_numeric.drop('MIS_Status', axis=1)  # 特徴量
y = train_numeric['MIS_Status']               # 目的変数

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 特徴量名をクリーンアップ
X_train.columns = X_train.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)
X_test.columns = X_test.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)

# LightGBMモデルの設定
lgb = lgb.LGBMClassifier()

# モデルの訓練
lgb.fit(X_train, y_train)

# モデルの評価
y_pred = lgb.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

[LightGBM] [Info] Number of positive: 30192, number of negative: 3653
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030770 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1006
[LightGBM] [Info] Number of data points in the train set: 33845, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.892067 -> initscore=2.112028
[LightGBM] [Info] Start training from score 2.112028
Accuracy: 0.8993145828409359


In [ ]:
from sklearn.metrics import f1_score

mean_f1 = f1_score(y_test, y_pred, average='macro')
print(f'Mean F1 Score: {mean_f1}')

Mean F1 Score: 0.5689823267559357


In [ ]:
predictions = lgb.predict(test[numeric_columns.drop('MIS_Status')] )
print(predictions)


[1 1 1 ... 1 1 0]


In [ ]:
sample_submit[1] = predictions
sample_submit.to_csv('/content/drive/MyDrive/fin_tech/submission1.csv',header=None)